<a href="https://colab.research.google.com/github/lcfdiniz/puc-rio/blob/main/real-estate-market/real_estate_market.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de dados do mercado imobiliário

por [Lucas Fiorani Diniz](https://www.linkedin.com/in/lcfdiniz/)

## Definição do problema

## Análise de dados

## Pré-processamento de dados

## Conclusões